<a href="https://colab.research.google.com/github/LoniQin/deep_learning_notebooks/blob/master/House_Price_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import dependencies

In [6]:
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, nn
import numpy as np
import pandas as pd

## Load data

In [7]:
url = "https://raw.githubusercontent.com/LoniQin/deep_learning_notebooks/master/"
train_data = pd.read_csv(url + 'data/kaggle_house_pred_train.csv')
test_data = pd.read_csv(url + 'data/kaggle_house_pred_test.csv')
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
# Standardization
all_features[numeric_features] = all_features[numeric_features].apply(lambda x : (x - x.mean()) / (x.std()))
all_features[numeric_features] = all_features[numeric_features].fillna(0)
# Change discrete numerical value to indicative value
all_features = pd.get_dummies(all_features, dummy_na=True)
n_train = train_data.shape[0]
train_featrues = nd.array(all_features[:n_train].values)
test_featrues = nd.array(all_features[n_train:].values)
train_labels = nd.array(train_data.SalePrice.values).reshape((-1, 1))

## Define constants

In [8]:
loss = gloss.L2Loss()
k, num_epochs, learning_rate, weight_decay, batch_size = 6, 100, 0.1, 0.1, 64

## Define model

In [9]:
def get_net():
    net = nn.Sequential()
    net.add(nn.Dense(10))
    net.add(nn.Dropout(0.01))
    net.add(nn.Dense(1))
    net.initialize()
    return net

## Define loss function




In [10]:
def log_rmse(net, features, labels):
    clipped_preds = nd.clip(net(features), 1, float('inf'))
    rmse = nd.sqrt(2 * loss(clipped_preds.log(), labels.log()).mean())
    return rmse.asscalar()

## Define traning function

In [11]:
def train(net, train_features, train_labels, test_features, test_labels, num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = gdata.DataLoader(gdata.ArrayDataset(train_features, train_labels), batch_size, shuffle=True)
    trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': learning_rate, 'wd': weight_decay})
    for epoch in range(num_epochs):
        for X, y in train_iter:
            with autograd.record():
                l = loss(net(X), y)
            l.backward()
            trainer.step(batch_size)
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls

## Define K Fold algorithm

In [12]:

def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part = X[idx, :]
        y_part = y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif  X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = nd.concat(X_train, X_part, dim = 0)
            y_train = nd.concat(y_train, y_part, dim = 0)
    return X_train, y_train, X_valid, y_valid

def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay, batch_size):
    train_l_sum , valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate, weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        #if i == 0:
           # utils.semilogy(range(1, num_epochs  + 1), train_ls, 'epochs', 'rmse', range(1, num_epochs  + 1), valid_ls, ['train', 'valid'])
        print("fold %d, train rmse %f, valid rmse %f" % (i, train_ls[-1], valid_ls[-1]))
    return train_l_sum / k, valid_l_sum / k




## Putting it all together

In [13]:
train_l, valid_l = k_fold(k, train_featrues, train_labels, num_epochs, learning_rate, weight_decay, batch_size)
print("%d-fold validation: avg train rmse %f, avg valid rmse %f" % (k, train_l, valid_l))

fold 0, train rmse 0.125407, valid rmse 0.153833
fold 1, train rmse 0.127392, valid rmse 0.142196
fold 2, train rmse 0.117930, valid rmse 0.163810
fold 3, train rmse 0.127066, valid rmse 0.130273
fold 4, train rmse 0.127552, valid rmse 0.150990
fold 5, train rmse 0.122720, valid rmse 0.164433
6-fold validation: avg train rmse 0.124678, avg valid rmse 0.150923
